## Muhammad Usman
### P19-0096

In [2]:
import re
from collections import Counter

# Load data.txt as corpus
with open('/data.txt', 'r') as f:
    corpus = f.read().lower()

# Create a vocabulary of words from the corpus
vocab = re.findall(r'\b\w+\b', corpus)
word_counts = Counter(vocab)

# Load common misspellings in Roman Urdu from misspellings.txt
with open('/misspellings.txt', 'r') as f:
    misspellings = f.readlines()
misspellings = [line.strip().split(',') for line in misspellings]



In [ ]:
# Define insert, delete, substitute and transpose tables using misspellings
insert_table = {}
delete_table = {}
substitute_table = {}
transpose_table = {}

for misspelling in misspellings:
    if len(misspelling[0]) < len(misspelling[1]):
        table = insert_table
        word = misspelling[1]
    elif len(misspelling[0]) > len(misspelling[1]):
        table = delete_table
        word = misspelling[1]
    else:
        if misspelling[0] != misspelling[1]:
            table = substitute_table
            word = misspelling[1]
        else:
            continue    
        
        if len(misspelling[0]) < 2:
            continue
        
        for i in range(len(misspelling[0])-1):
            transposed_word = misspelling[0][:i] + misspelling[0][i+1] + misspelling[0][i] + misspelling[0][i+2:]
            if transposed_word == misspelling[1]:
                table = transpose_table
                word = misspelling[1]
                break
    
    table[word] = table.get(word, 0) + 1


In [4]:
# Define a function to calculate P(x|w) using the error model tables
def calculate_P_x_given_w(x, w):
    # initialize probabilities with 1
    P_insert = P_delete = P_substitute = P_transpose = 1

    # calculate probabilities for insertions
    for i in range(len(x)):
        x_insert = x[:i] + ' ' + x[i:]
        if x_insert in insert_table and w in insert_table[x_insert]:
            P_insert *= insert_table[x_insert][w] / word_counts[x_insert]

    # calculate probabilities for deletions
    for i in range(len(x)):
        x_delete = x[:i] + x[i+1:]
        if x_delete in delete_table and w in delete_table[x_delete]:
            P_delete *= delete_table[x_delete][w] / word_counts[x_delete]

    # calculate probabilities for substitutions
    for i in range(len(x)):
        for c in 'abcdefghijklmnopqrstuvwxyz':
            x_substitute = x[:i] + c + x[i+1:]
            if x_substitute in substitute_table and w in substitute_table[x_substitute]:
                P_substitute *= substitute_table[x_substitute][w] / word_counts[x_substitute]
                
    # calculate probabilities for transpositions
    for i in range(len(x)-1):
        x_transpose = x[:i] + x[i+1] + x[i] + x[i+2:]
        if x_transpose in transpose_table and w in transpose_table[x_transpose]:
            P_transpose *= transpose_table[x_transpose][w] / word_counts[x_transpose]
    P_x_given_w = P_insert * P_delete * P_substitute * P_transpose
    return P_x_given_w


In [5]:
def get_candidate_words(x, V):
    candidate_words = set()
    
    # insertions
    for i in range(len(x) + 1):
        for c in 'abcdefghijklmnopqrstuvwxyz':
            candidate = x[:i] + c + x[i:]
            if candidate in V:
                candidate_words.add(candidate)
    
    # deletions
    for i in range(len(x)):
        candidate = x[:i] + x[i+1:]
        if candidate in V:
            candidate_words.add(candidate)
    
    # substitutions
    for i in range(len(x)):
        for c in 'abcdefghijklmnopqrstuvwxyz':
            candidate = x[:i] + c + x[i+1:]
            if candidate != x and candidate in V:
                candidate_words.add(candidate)
    
    # transpositions
    for i in range(len(x) - 1):
        candidate = x[:i] + x[i+1] + x[i] + x[i+2:]
        if candidate in V:
            candidate_words.add(candidate)
    
    return candidate_words


In [7]:
def calculate_P_w(w, corpus):
    # create a vocabulary of words from the corpus
    vocab = re.findall(r'\b\w+\b', corpus.lower())
    # count the number of occurrences of each word in the vocabulary
    word_counts = Counter(vocab)
    # calculate the total number of words in the corpus
    total_words = sum(word_counts.values()) 
    # calculate the probability of the word w
    P_w = word_counts[w] / total_words
    return P_w

In [12]:
x = 'suay'
candidate_words = get_candidate_words(x, corpus)

if not candidate_words:
    # handle the case where there are no candidate words
    print("No candidate words found for '{}'".format(x))
else:
    P_x_given_w = {}
    for w in candidate_words:
        P_x_given_w[w] = calculate_P_x_given_w(x, w) # function to calculate P(x|w)

    # calculate P(w) for each candidate word w
    P_w = {}
    for w in candidate_words:
        P_w[w] = calculate_P_w(w,corpus) # function to calculate P(w)

    # calculate the score for each candidate word w
    scores = {}
    for w in candidate_words:
        scores[w] = P_x_given_w[w] * P_w[w]

    # select the candidate word with the highest score
    selected_word = max(scores, key=scores.get)
    print("Corrected word for '{}' is '{}'".format(x, selected_word))


Corrected word for 'suay' is 'say'
